In [22]:
# Imports (non-exhaustive)
import numpy as np    
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
# from tensorflow.keras.layers import RandomFlip, RandomZoom, RandomRotation, Rescaling
from tensorflow.python.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# .python must be added for tensorflow versions 1.xx

In [4]:
# Directories definition
train_path = "C:/Users/user/Desktop/Chest Xray Classifier/dataset/train"
test_path = "C:/Users/user/Desktop/Chest Xray Classifier/dataset/test"
val_path = "C:/Users/user/Desktop/Chest Xray Classifier/dataset/val"

# Basic parameters (image dimension and batch size)
batch_size = 16
img_height = 500
img_width = 500

In [5]:
# Data Preparation

# Load data

train = tf.keras.utils.image_dataset_from_directory(train_path,
                                                    image_size=(img_height, img_width),
                                                    color_mode='grayscale',
                                                    batch_size=batch_size)

test = tf.keras.utils.image_dataset_from_directory(test_path,
                                                   image_size=(img_height, img_width),
                                                   color_mode='grayscale',
                                                   shuffle=False, 
                                                   label_mode='binary',
                                                   batch_size=batch_size)

valid = tf.keras.utils.image_dataset_from_directory(val_path,
                                                    image_size=(img_height, img_width),
                                                    color_mode='grayscale',
                                                    label_mode='binary',
                                                    batch_size=batch_size)


Found 4192 files belonging to 2 classes.
Found 624 files belonging to 2 classes.
Found 1040 files belonging to 2 classes.


In [11]:
# Model Architecture (Input->CNN->Flat->ANN->Output)
cnn = Sequential()

# Preprocessing (data augmentation) layers
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomRotation(0.2),    
    ])
cnn.add(data_augmentation)

# Convolution and Pooling Layers
cnn.add(Conv2D(32, (3, 3), activation="relu", input_shape=(img_width, img_height, 1)))
cnn.add(MaxPooling2D(pool_size=(2, 2)))
cnn.add(Conv2D(32, (3, 3), activation="relu", input_shape=(img_width, img_height, 1)))
cnn.add(MaxPooling2D(pool_size=(2, 2)))
cnn.add(Conv2D(64, (3, 3), activation="relu", input_shape=(img_width, img_height, 1)))
cnn.add(MaxPooling2D(pool_size=(2, 2)))
cnn.add(Conv2D(64, (3, 3), activation="relu", input_shape=(img_width, img_height, 1)))
cnn.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten 
cnn.add(Flatten())

# Fully-connected Neural Network Layers
cnn.add(Dense(activation='relu', units = 128))
cnn.add(Dense(activation='relu', units = 64))
cnn.add(Dense(activation='relu', units = 64))
cnn.add(Dense(activation='sigmoid', units = 1))

cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Regularization callbacks
early = EarlyStopping(monitor="val_loss", mode="min", patience=3) # Patience: no. of epochs to run after monitored parameter stops changing
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1, factor=0.3, min_lr=0.00001)
callbacks_ls = [early, learning_rate_reduction] 

# Class weights for unbalanced datasets
# Assign higher weights to the minority class, reduce bias towards majority class
# Calculate proportion, invert it as a counter-bias
# from sklearn.utils.class_weight import compute_class_weight
# weights = compute_class_weight('balanced', np.unique(train.class_names))
# class_weight_val = dict(zip(np.unique(train.class_names), weights))
# #print(class_weight_val)

In [ ]:
# Train model (run this cell to train)
# cnn.fit(train, epochs=30, validation_data=valid, class_weight=class_weight_val, callbacks=callbacks_ls)
cnn.fit(train, epochs=30, validation_data=valid, callbacks=callbacks_ls)
# Training metric visualization
pd.DataFrame(cnn.history.history).plot()

In [ ]:
# Model evaluation
test_model = cnn.evaluate(test)
test_accu = test_model[1] * 100
print('Test Accuracy: ', test_accu, '%')

# Convert prediction output to binary classes (Sigmoid: 0 < output < 1)
prediction = cnn.predict(test, verbose=1)
prediction_bin = prediction.copy()

for i in prediction_bin:
    if (i <= 0.5):
        prediction_bin[i] = 0
    elif (i > 0.5):
        prediction_bin[i] = 1

# Confusion matrix
from sklearn.metrics import classification_report, confusion_matrix
con_mat = pd.DataFrame(data=confusion_matrix(test.classes, prediction_bin, labels=[0, 1]),
                       index=["Actual Normal", "Actual Pneumonia"],
                       columns=["Predicted Normal", "Predicted Pneumonia"])
print(classification_report(y_true=test.classes, y_pred=prediction_bin, target_names=['NORMAL','PNEUMONIA']))